# This notebook is a improved version of the PDF pre-processing of llama notebook

In this notbook we implement our knowlegde on CoT, RAG and agentic systems for the procesing infomation o the documents.

* Use a PDF (for now only text)
* Use `groq-llama-3.2-3b-preview` for the proces of the file
* Re write the PDF information on to a .txt for latter processing.

For now the first test will be on the `Attention is all you need paper`.

In [26]:
from typing import Optional
import os
from groq import Groq
from groq import Client

from tqdm.notebook import tqdm 
# from llama_index.llms.groq import Groq


from dotenv import load_dotenv 
load_dotenv()
import warnings
warnings.filterwarnings("ignore")

Now let's check the pdf_path the name of the model we will use and declare the global variable


In [33]:
doc_path = "/home/mau/Documents/Projects/podcast_llama/pdf_docs/Scaling Laws for Neural Language Modelspdf.pdf"
model_1 = "llama-3.2-3b-preview"

api_key=os.getenv("GROQ_API_KEY")

In [28]:
print(api_key)

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


## We now use llamaindex for parsing and extracting the document information. 
We have 2 ways of calling the model, 
1. Directly with groq:
```python
            from groq import Groq

            client = Groq(
                api_key=api_key,
                model=model_1,
            )
```

2. Using llamaidex 
```python
            from llama_index.llms.groq import Groq
            
            llm = Groq(model=model_1, api_key=api_key)  
```

3. Using Client from groq-> `This is what we are going to implement`
```python
            from groq import Client

            client = Client(
                api_key=api_key
                )
```

In [29]:
client = Client(
    api_key=api_key
)

this part is exactly the same as in the original notebook llama, this is to make sure the file exist

In [34]:
def validate_pdf(file_path: str) -> bool:
    if not os.path.exists(file_path):
        print(f"Error: File not found at path: {file_path}")
        return False
    if not file_path.lower().endswith('.pdf'):
        print("Error: File is not a PDF")
        return False
    return True

In [35]:
validate_pdf(doc_path)

True


## Let's do the retrieval of the information. 

We can use the `llamaindex` framework for this purpose. We will retrieve the information of the document, it can contain text, latex or tables and we want to read it all. 

The main differences with this proposition and metaAI is that we are using a LLM provider and a framework, we believe that this approach is more reliable and efficient

In [ ]:
from typing import Dict, Tuple, Optional
from PyPDF2 import PdfReader

In [ ]:
def process_pdf(file_path: str, max_chars: int ) -> Optional[Tuple[Dict, str]]:
    """
    Process a PDF file to extract metadata and content with page tracking.
    
    Args:
        file_path: Path to the PDF file
        max_chars: Maximum number of characters to extract (default: 100000)
        
    Returns:
        Tuple containing metadata dictionary and extracted text, or None if validation fails
    """
    if not validate_pdf(file_path):
        return None

    try:
        print("\nExtracting metadata...")
        
        try:
            pdf = PdfReader(file_path)
            if not pdf.pages:
                print("Error: PDF appears to be empty or corrupted")
                return None
            total_pages = len(pdf.pages)
        except Exception as e:
            print(f"Error reading PDF with pypdf: {str(e)}")
            return None
            
        doc_info = {
            "Pages": total_pages,
            "Metadata": pdf.metadata
        }
        
        print("PDF Metadata:")
        print(f"Number of pages: {total_pages}")
        print("Document info:", doc_info['Metadata'])
        
        print("\nExtracting text...")
        print(f"Starting PDF processing (limit: {max_chars} characters)...")
        
        try:
            current_length = 0
            text_content = ""
            pages_processed = 0
            
            for page_num in range(total_pages):
                if current_length >= max_chars:
                    break
                    
                current_page = pdf.pages[page_num]
                page_text = current_page.extract_text()
                
                remaining_chars = max_chars - current_length
                if len(page_text) <= remaining_chars:
                    text_content += page_text
                    current_length += len(page_text)
                    pages_processed += 1
                    print(f"Processed page {page_num + 1}/{total_pages} (Characters: {current_length}/{max_chars})")
                else:
                    text_content += page_text[:remaining_chars]
                    current_length += remaining_chars
                    pages_processed += 1
                    print(f"Processed page {page_num + 1}/{total_pages} (Reached character limit of {max_chars})")
                    break
            
            # Save extracted text to file
            output_file = "extracted_text.txt"
            with open(output_file, "w", encoding="utf-8") as f:
                f.write(text_content)
            
            print(f"\nProcessing complete:")
            print(f"- Pages processed: {pages_processed} out of {total_pages}")
            print(f"- Characters extracted: {current_length}")
            if pages_processed < total_pages:
                print(f"- Stopped at page {pages_processed} due to character limit")
            print(f"- Extracted text saved to {output_file}")
            
            return doc_info, text_content
            
        except Exception as e:
            print(f"Error extracting text: {str(e)}")
            return None
            
    except Exception as e:
        print(f"\nError processing PDF: {str(e)}")
        return None

In [77]:
result = process_pdf(doc_path, max_chars=8000)



Extracting metadata...
PDF Metadata:
Number of pages: 30
Document info: {'/Author': '', '/CreationDate': 'D:20200124010539Z', '/Creator': 'LaTeX with hyperref package', '/Keywords': '', '/ModDate': 'D:20200124010539Z', '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', '/Producer': 'pdfTeX-1.40.17', '/Subject': '', '/Title': '', '/Trapped': '/False'}

Extracting text...
Starting PDF processing (limit: 8000 characters)...
Processed page 1/30 (Characters: 1647/8000)
Processed page 2/30 (Characters: 3531/8000)
Processed page 3/30 (Characters: 7089/8000)
Processed page 4/30 (Reached character limit of 8000)

Processing complete:
- Pages processed: 4 out of 30
- Characters extracted: 8000
- Stopped at page 4 due to character limit
- Extracted text saved to extracted_text.txt


In [40]:
from llama_index.core import (
    SimpleDirectoryReader,
)
from llama_index.core.text_splitter import SentenceSplitter




In [ ]:
CLEANING_PROMPT = """
You are a world class text pre-processor, here is the raw data from a PDF, please parse and return it in a way that is crispy and usable to send to a podcast writer.

The raw data is messed up with new lines, Latex math and you will see fluff that we can remove completely. Basically take away any details that you think might be useless in a podcast author's transcript.

Remember, the podcast could be on any topic whatsoever so the issues listed above are not exhaustive

Please be smart with what you remove and be creative ok?

Remember DO NOT START SUMMARIZING THIS, YOU ARE ONLY CLEANING UP THE TEXT AND RE-WRITING WHEN NEEDED

Be very smart and aggressive with removing details, you will get a running portion of the text and keep returning the processed text.

PLEASE DO NOT ADD MARKDOWN FORMATTING, STOP ADDING SPECIAL CHARACTERS THAT MARKDOWN CAPITALIZATION ETC LIKES

ALWAYS start your response directly with processed text and NO ACKNOWLEDGEMENTS about my questions ok?
Here is the text:
"""


In [42]:
def clean_text_chunks(file_path: str, chunk_size: int = 16):
    """
    Load text, chunk it, and clean each chunk using Groq
    """
    # Load the document
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    
    # Initialize text splitter
    text_splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=4)
    
    # Split text into chunks
    chunks = text_splitter.split_text(documents[0].text)
    print(f"Split text into {len(chunks)} chunks")
    
     # Clean each chunk
    cleaned_text = ""
    for chunk in tqdm(chunks, desc="Cleaning chunks"):
        try:
            # Use Groq's chat completions
            completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": CLEANING_PROMPT
                    },
                    {
                        "role": "user",
                        "content": chunk
                    }
                ],
                model=model_1,
                temperature=0.7,
                max_tokens=1024
            )
            
            # Extract cleaned text
            cleaned_chunk = completion.choices[0].message.content
            cleaned_text += cleaned_chunk + "\n"
            
        except Exception as e:
            print(f"Error processing chunk: {e}")
            continue
    # Save cleaned text
    output_file = "cleaned_text.txt"
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(cleaned_text)
    
    print(f"\nCleaned text saved to {output_file}")
    return cleaned_text

In [43]:
# Use the function
cleaned_text = clean_text_chunks("/home/mauricio/Documents/Projects/podcast_llama/extracted_text.txt")

Split text into 157 chunks


Cleaning chunks:   0%|          | 0/157 [00:00<?, ?it/s]


Cleaned text saved to cleaned_text.txt
